In [1]:
import os
import sys
sys.path.append("../cnn")
sys.path.append("../utils")

import torch
from torch.autograd import Variable

from ipywidgets import interact, fixed
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# import seaborn as sns; sns.set()

from cnn_md import CNNMultidecoder, CNNVariationalMultidecoder
from hao_data import HaoEvalDataset

In [6]:
# Set up environment variables for the model we want to examine
# NOT necessarily the current environment variables!!

feat_dim=80
left_splice=5
right_splice=5

optimizer="Adam"
learning_rate="0.0001" # Use string instead of float to prevent Python's auto-formatting...
epochs=25
batch_size=256

enc_channels=[256, 256]
enc_kernels=[3, 3]
enc_pools=[3, 3]
enc_fc=[2048]

latent_dim=1024

dec_fc=[2048]
dec_channels=[256, 256]
dec_kernels=[3, 3]
dec_pools=[3, 3]

use_batch_norm=False
use_batch_norm_str = "true" if use_batch_norm else "false"
activation="SELU"
weight_init="xavier_uniform"

enc_channels_delim="_" + "_".join(map(str, enc_channels))
if len(enc_channels) == 0:
     # need to recreate bash join behavior with empty array
    enc_channels_delim="_"
enc_kernels_delim="_" + "_".join(map(str, enc_kernels))
if len(enc_kernels) == 0:
     # need to recreate bash join behavior with empty array
    enc_kernels_delim="_"
enc_pools_delim="_" + "_".join(map(str, enc_pools))
if len(enc_pools) == 0:
     # need to recreate bash join behavior with empty array
    enc_pools_delim="_"
enc_fc_delim="_" + "_".join(map(str, enc_fc))
if len(enc_fc) == 0:
     # need to recreate bash join behavior with empty array
    enc_fc_delim="_"
    
dec_fc_delim="_" + "_".join(map(str, dec_fc))
if len(dec_fc) == 0:
     # need to recreate bash join behavior with empty array
    dec_fc_delim="_"
dec_channels_delim="_" + "_".join(map(str, dec_channels))
if len(dec_channels) == 0:
     # need to recreate bash join behavior with empty array
    dec_channels_delim="_"
dec_kernels_delim="_" + "_".join(map(str, dec_kernels))
if len(dec_kernels) == 0:
     # need to recreate bash join behavior with empty array
    dec_kernels_delim="_"
dec_pools_delim="_" + "_".join(map(str, dec_pools))
if len(dec_pools) == 0:
     # need to recreate bash join behavior with empty array
    dec_pools_delim="_"

debug_model = False
debug_str = "true" if debug_model else "false"
expt_name = "ENC_C%s_K%s_P%s_F%s/LATENT_%d/DEC_F%s_C%s_K%s_P%s/ACT_%s_BN_%s_WEIGHT_INIT_%s/OPT_%s_LR_%s_EPOCHS_%d_BATCH_%d_DEBUG_%s" % (enc_channels_delim,
                                                                                                                                        enc_kernels_delim, 
                                                                                                                                        enc_pools_delim, 
                                                                                                                                        enc_fc_delim,
                                                                                                                                        latent_dim,
                                                                                                                                        dec_fc_delim,
                                                                                                                                        dec_channels_delim,
                                                                                                                                        dec_kernels_delim,                                                                                                                   dec_pools_delim,
                                                                                                                                        activation,
                                                                                                                                        use_batch_norm_str,
                                                                                                                                        weight_init,
                                                                                                                                        optimizer,
                                                                                                                                        learning_rate,
                                                                                                                                        epochs,
                                                                                                                                        batch_size,
                                                                                                                                        debug_str)

time_dim = (left_splice + right_splice + 1)
freq_dim = feat_dim

noise_ratio=0.25
print("Noise ratio %s" % str(noise_ratio))

print("Using experiment %s" % expt_name)

Noise ratio 0.25
Using experiment ENC_C_256_256_K_3_3_P_3_3_F_2048/LATENT_1024/DEC_F_2048_C_256_256_K_3_3_P_3_3/ACT_SELU_BN_false_WEIGHT_INIT_xavier_uniform/OPT_Adam_LR_0.0001_EPOCHS_25_BATCH_256_DEBUG_false


In [7]:
# Set up datasets for IHM, SDM1 baselines (dev set only)
dataset_name = "ami-0.1"
current_feats = "/data/sls/r/u/atitus5/meng/%s" % dataset_name
ihm_baseline = HaoEvalDataset(os.path.join(current_feats, "ihm-dev-norm.blogmel.scp"))
sdm1_baseline = HaoEvalDataset(os.path.join(current_feats, "sdm1-dev-norm.blogmel.scp"))
print("Set up baseline dev datasets")

Set up baseline dev datasets


In [8]:
# Set up augmented datasets for IHM->IHM, IHM->SDM1, SDM1->SDM1, SDM1->IHM (dev set only)
model_type = "ae"
augmented_data_dir = "/data/sls/scratch/atitus5/meng/augmented_data/cnn/%s/%s_ratio%s" % (expt_name,
                                                                                          model_type,
                                                                                          str(noise_ratio))
'''
augmented_data_dir = "/data/sls/scratch/atitus5/meng/augmented_data/cnn/%s/%s" % (expt_name,
                                                                                  model_type)
'''

ihm_ihm = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_ihm-tar_ihm.scp"))
ihm_sdm1 = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_ihm-tar_sdm1.scp"))
sdm1_sdm1 = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_sdm1-tar_sdm1.scp"))
sdm1_ihm = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_sdm1-tar_ihm.scp"))
print("Set up augmented dev datasets")

Set up augmented dev datasets


In [10]:
# color_map = "coolwarm"
color_map = "viridis"
def plotParallelUtts(utt_id_idx):
    fig, axarr = plt.subplots(6, sharex=True)
    
    fig.set_size_inches(8.5, 11)
    
    # IHM baseline
    ihm_baseline_utt_id = ihm_baseline.utt_ids[utt_id_idx]
    ihm_baseline_feats = np.transpose(ihm_baseline.feats_for_uttid(ihm_baseline_utt_id))
    print(ihm_baseline_feats.shape[1])
    axarr[0].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[0].imshow(ihm_baseline_feats, origin='lower', cmap=color_map, aspect='auto')
    axarr[0].set_title("IHM")
    
    # SDM1 baseline
    sdm1_baseline_utt_id = sdm1_baseline.utt_ids[utt_id_idx]
    sdm1_baseline_feats = np.transpose(sdm1_baseline.feats_for_uttid(sdm1_baseline_utt_id))
    axarr[1].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[1].imshow(sdm1_baseline_feats, origin='lower', cmap=color_map, aspect='auto')
    axarr[1].set_title("SDM1")
    
    # IHM->IHM baseline
    ihm_ihm_utt_id = ihm_ihm.utt_ids[utt_id_idx]
    ihm_ihm_feats = np.transpose(ihm_ihm.feats_for_uttid(ihm_ihm_utt_id))
    axarr[2].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[2].imshow(ihm_ihm_feats, origin='lower', cmap=color_map, aspect='auto')
    axarr[2].set_title("IHM>IHM")
    
    # IHM->SDM1 baseline
    ihm_sdm1_utt_id = ihm_sdm1.utt_ids[utt_id_idx]
    ihm_sdm1_feats = np.transpose(ihm_sdm1.feats_for_uttid(ihm_sdm1_utt_id))
    axarr[3].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[3].imshow(ihm_sdm1_feats, origin='lower', cmap=color_map, aspect='auto')
    axarr[3].set_title("IHM>SDM1")
    
    # SDM1->SDM1 baseline
    sdm1_sdm1_utt_id = sdm1_sdm1.utt_ids[utt_id_idx]
    sdm1_sdm1_feats = np.transpose(sdm1_sdm1.feats_for_uttid(sdm1_sdm1_utt_id))
    axarr[4].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[4].imshow(sdm1_sdm1_feats, origin='lower', cmap=color_map, aspect='auto')
    axarr[4].set_title("SDM1>SDM1")
    
    # SDM1->IHM baseline
    sdm1_ihm_utt_id = sdm1_ihm.utt_ids[utt_id_idx]
    sdm1_ihm_feats = np.transpose(sdm1_ihm.feats_for_uttid(sdm1_ihm_utt_id))
    axarr[5].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[5].imshow(sdm1_ihm_feats, origin='lower', cmap=color_map, aspect='auto')
    axarr[5].set_title("SDM1>IHM")
    
    plt.tight_layout()
    
    # fig.savefig("Utt Idx %d" % utt_id_idx, dpi=1024)
    
    plt.show()

interact(plotParallelUtts, utt_id_idx=range(len(ihm_baseline)))

A Jupyter Widget

<function __main__.plotParallelUtts>